<a href="https://colab.research.google.com/github/nasremp/PRID/blob/main/pre_trained_densenet201_ReID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/KaiyangZhou/deep-person-reid.git

fatal: destination path 'deep-person-reid' already exists and is not an empty directory.


In [2]:
%cd deep-person-reid/

/content/deep-person-reid


In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [4]:
!conda create --name torchreid python=3.9

Solving environment: \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.14.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /usr/local/envs/torchreid

  added / updated specs:
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |            2_gnu          23 KB  conda-forge
    ca-certificates-2022.6.15  |       ha878542_0         149 KB  conda-forge
    ld_impl_linux-64-2.36.1    |       hea4e1c9_2         667 KB  conda-forge
    libffi-3.4.2               |       h7f98852_5          57 KB  conda-forge
    libgcc-ng-12.1.0           |      h8d9b700_16         940 KB  conda-forge
    libgomp-12.1.0             |      h8d9b700_16         459 KB  conda-forge
    libnsl-2.0.0               |       h7f98852_0          31 KB

In [ ]:
! conda activate torchreid


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 26.9 MB/s 
     |████████████████████████████████| 1.9 MB 52.6 MB/s 
     |████████████████████████████████| 4.1 MB 88.6 MB/s 
     |████████████████████████████████| 3.1 MB 62.8 MB/s 
     |████████████████████████████████| 38.1 MB 434 kB/s 
     |████████████████████████████████| 60.9 MB 1.2 MB/s 
     |████████████████████████████████| 11.2 MB 61.2 MB/s 
     |████████████████████████████████| 5.9 MB 52.1 MB/s 
     |████████████████████████████████| 829 kB 58.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61 kB 467 kB/s 
     |████████████████████████████████| 190 kB 76.0 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 3.4 MB 36.8 MB/s 
     |█████████████████

In [5]:
!conda install pytorch torchvision cudatoolkit=9.0 -c pytorch

Solving environment: \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - cudatoolkit=9.0

Current channels:

  - https://conda.anaconda.org/pytorch/linux-64
  - https://conda.anaconda.org/pytorch/noarch
  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [6]:
import torchreid
import torch 
print(torch.__version__)

1.12.1+cu113


/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '


## Available PRID models

In [7]:
torchreid.models.show_avai_models()

['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'osnet_ain_x0_75', 'osnet_ain_x0_5', 'osnet_ain_x0_25']


**Load data manager**

---



In [9]:
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources="market1501",
    targets="market1501",
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 2795 KB/s, 53 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




**Build model, optimizer and lr_scheduler**

In [10]:
model = torchreid.models.build_model(
    name="densenet201",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth


  0%|          | 0.00/77.4M [00:00<?, ?B/s]

In [11]:
torchreid.utils.load_pretrained_weights(model, '/content/model.pth.tar-30')

Successfully loaded pretrained weights from "/content/model.pth.tar-30"


In [12]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [13]:
engine.run(
    save_dir="log/densenet201",
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    visrank=True,
    visrank_topk=3,
    test_only=True
)

##### Evaluating market1501 (source) #####
Extracting features from query set ...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Done, obtained 3368-by-1920 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-1920 matrix
Speed: 0.1472 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 66.1%
CMC curve
Rank-1  : 84.7%
Rank-5  : 92.8%
Rank-10 : 95.0%
Rank-20 : 96.7%
# query: 3368
# gallery 15913
Visualizing top-3 ranks ...
- done 100/3368
- done 200/3368
- done 300/3368
- done 400/3368
- done 500/3368
- done 600/3368
- done 700/3368
- done 800/3368
- done 900/3368
- done 1000/3368
- done 1100/3368
- done 1200/3368
- done 1300/3368
- done 1400/3368
- done 1500/3368
- done 1600/3368
- done 1700/3368
- done 1800/3368
- done 1900/3368
- done 2000/3368
- done 2100/3368
- done 2200/3368
- done 2300/3368
- done 2400/3368
- done 2500/3368
- done 2600/3368
- done 2700/3368
- done 2800/3368
- done 2900/3368
- done 3000/3368
- done 3100/3368
- done 3200/3368
- done 3300/3368
Done. Images have been saved to "log/densenet201/visrank_market1501" ...
